# Runtime Arguments 바인딩

때로는 Runnable 시퀀스 내에서 Runnable을 호출할 때, 이전 Runnable의 출력이나 사용자 입력에 포함되지 않은 상수 인자를 전달해야 할 경우가 있습니다.

이럴 때는 `Runnable.bind()`를 사용하면 이러한 인자를 쉽게 전달할 수 있습니다.


In [1]:
%pip install -qU langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


`RunnablePassthrough`를 사용하여 `{equation_statement}` 변수를 프롬프트에 전달하고, `StrOutputParser`를 사용하여 모델의 출력을 문자열로 파싱하는 `runnable` 객체를 생성합니다.

- `runnable.invoke()` 메서드를 호출하여 "x raised to the third plus seven equals 12"라는 방정식 문장을 전달하고 결과를 출력합니다.


In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # 대수 기호를 사용하여 다음 방정식을 작성한 다음 풀이하세요.
            "Write out the following equation using algebraic symbols then solve it. "
            "Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        (
            "human",
            "{equation_statement}",  # 사용자가 입력한 방정식 문장을 변수로 받습니다.
        ),
    ]
)
# ChatOpenAI 모델을 초기화하고 temperature를 0으로 설정합니다.
model = ChatOpenAI(temperature=0)

# 방정식 문장을 입력받아 프롬프트에 전달하고, 모델에서 생성된 결과를 문자열로 파싱합니다.
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

# 예시 방정식 문장을 입력하여 결과를 출력합니다.
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION: 
Subtract 7 from both sides:
x^3 = 5

Take the cube root of both sides:
x = ∛5


그리고 특정 `stop` 단어를 사용하여 모델을 호출하고자 합니다. `model.bind()`를 사용하여 언어 모델을 호출하고, 생성된 텍스트에서 "SOLUTION" 토큰까지만 출력합니다.


In [3]:
runnable = (
    # 실행 가능한 패스스루 객체를 생성하여 "equation_statement" 키에 할당합니다.
    {"equation_statement": RunnablePassthrough()}
    | prompt  # 프롬프트를 파이프라인에 추가합니다.
    | model.bind(
        stop="SOLUTION"
    )  # 모델을 바인딩하고 "SOLUTION" 토큰에서 생성을 중지하도록 설정합니다.
    | StrOutputParser()  # 문자열 출력 파서를 파이프라인에 추가합니다.
)
# "x raised to the third plus seven equals 12"라는 입력으로 파이프라인을 실행하고 결과를 출력합니다.
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12




## OpenAI 함수 연결하기

바인딩의 특히 유용한 활용 방법 중 하나는 호환되는 OpenAI 모델에 OpenAI 함수를 연결하는 것입니다.


- `function` 객체는 방정식을 수식화하고 해결하는 기능을 나타냅니다.
- `name` 속성은 함수의 이름인 "solver"로 설정되어 있습니다.
- `description` 속성은 함수의 설명으로, "Formulates and solves an equation"이라고 명시되어 있습니다.
- `parameters` 속성은 함수의 매개변수를 정의하는 객체입니다.
  - `type` 속성은 매개변수의 타입이 "object"임을 나타냅니다.
  - `properties` 속성은 매개변수의 세부 속성을 정의합니다.
    - `equation` 속성은 문자열 타입이며, 방정식의 대수식을 나타냅니다.
    - `solution` 속성은 문자열 타입이며, 방정식의 해를 나타냅니다.
  - `required` 속성은 필수 매개변수로 `equation`과 `solution`이 포함되어야 함을 나타냅니다.


In [4]:
function = {
    "name": "solver",  # 함수의 이름
    # 함수의 설명: 방정식을 수립하고 해결합니다.
    "description": "Formulates and solves an equation",
    "parameters": {  # 함수의 매개변수
        "type": "object",  # 매개변수의 타입: 객체
        "properties": {  # 매개변수의 속성
            "equation": {  # 방정식 속성
                "type": "string",  # 방정식의 타입: 문자열
                "description": "The algebraic expression of the equation",  # 방정식의 대수식 표현
            },
            "solution": {  # 해답 속성
                "type": "string",  # 해답의 타입: 문자열
                "description": "The solution to the equation",  # 방정식의 해답
            },
        },
        "required": ["equation", "solution"],  # 필수 매개변수: 방정식과 해답
    },
}

- `ChatPromptTemplate`을 사용하여 프롬프트 템플릿을 정의합니다.
  - "system" 역할에는 대수 기호를 사용하여 방정식을 작성하고 풀라는 지시를 할당합니다.
  - "human" 역할에는 `{equation_statement}` 변수를 사용하여 실제 방정식 문장을 받습니다.
- `bind()` 메서드를 사용하여 `solver`라는 이름의 함수 호출을 모델에 바인딩합니다.
- `RunnablePassthrough`를 사용하여 `equation_statement` 변수를 정의하고, 프롬프트 템플릿과 모델을 파이프라인으로 연결합니다.
- `invoke()` 메서드를 호출하여 "x raised to the third plus seven equals 12"라는 방정식 문장을 전달하고 모델에서 답을 얻습니다.


In [5]:
# 다음 방정식을 대수 기호를 사용하여 작성한 다음 해결하세요.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0).bind(
    function_call={"name": "solver"},  # funcion schema를 바인딩합니다.
    functions=[function],
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
# x의 세제곱에 7을 더하면 12와 같다
runnable.invoke("x raised to the third plus seven equals 12")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"equation":"x^3 + 7 = 12","solution":"x = \\\\sqrt[3]{5}"}', 'name': 'solver'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 98, 'total_tokens': 124}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': 'fp_a7daf7c51e', 'finish_reason': 'stop', 'logprobs': None})

## OpenAI 도구 연결하기

OpenAI에서 제공하는 도구를 연결하여 활용하는 방법에 대해 설명하겠습니다.

우선, OpenAI의 API 키를 발급받아 환경 변수로 설정해야 합니다. 그 다음, openai 모듈을 임포트하고 tool 객체를 생성합니다.

tool 객체는 OpenAI의 다양한 기능을 간편하게 사용할 수 있도록 도와줍니다.

예를 들어, `tool.run` 메서드를 호출하여 자연어로 된 질문을 입력하면, 해당 질문에 대한 답변을 생성해 줍니다.


- `tools` 리스트에는 하나의 딕셔너리 요소가 포함되어 있습니다.
- 이 딕셔너리는 "type"과 "function"이라는 두 개의 키를 가지고 있습니다.
- "type" 키의 값은 "function"으로 설정되어 있습니다.
- "function" 키의 값은 또 다른 딕셔너리로, "name", "description", "parameters"라는 키를 가지고 있습니다.
- "name" 키의 값은 `get_current_weather`로, 현재 날씨를 가져오는 함수의 이름을 나타냅니다.
- "description" 키의 값은 주어진 위치의 현재 날씨를 가져오는 기능에 대한 설명입니다.
- "parameters" 키의 값은 함수의 매개변수를 정의하는 또 다른 딕셔너리입니다.
  - "type"은 "object"로 설정되어 있습니다.
  - "properties"는 "location"과 "unit"이라는 두 개의 키를 가진 딕셔너리입니다.
    - "location"은 문자열 타입이며, 도시와 주(예: San Francisco, CA)를 나타냅니다.
    - "unit"은 문자열 타입이며, "celsius" 또는 "fahrenheit" 중 하나의 값을 가질 수 있습니다.
  - "required"는 ["location"]이 필수 매개변수임을 나타냅니다.


In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",  # 현재 날씨를 가져오는 함수의 이름
            "description": "주어진 위치의 현재 날씨를 가져옵니다",  # 함수에 대한 설명
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시와 주, 예: San Francisco, CA",  # 위치 매개변수에 대한 설명
                    },
                    # 온도 단위 매개변수 (섭씨 또는 화씨)
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],  # 필수 매개변수 지정
            },
        },
    }
]

- `bind()` 메서드를 사용하여 `tools`를 모델에 바인딩합니다.
- `invoke()` 메서드를 호출하여 "What's the weather in SF, NYC and LA?"라는 질문을 모델에 전달합니다.


In [7]:
# ChatOpenAI 모델을 초기화하고 도구를 바인딩합니다.
model = ChatOpenAI(model="gpt-3.5-turbo").bind(tools=tools)
# 모델을 호출하여 샌프란시스코, 뉴욕, 로스앤젤레스의 날씨에 대해 질문합니다.
model.invoke("What's the weather in SF, NYC and LA?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zLUq1xzrmSS6Ru1gk9J4JGie', 'function': {'arguments': '{"location": "San Francisco, CA"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_uyMD7nMJvVSaZsu6RxI7DHLT', 'function': {'arguments': '{"location": "New York, NY"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_MhK8kclBTRfggZ5bWdG7jxYV', 'function': {'arguments': '{"location": "Los Angeles, CA"}', 'name': 'get_current_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 94, 'total_tokens': 163}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'tool_calls', 'logprobs': None})